In [42]:
import pandas
import os.path

In [62]:
def getData(fromCache=True):
    # Retrieved 13-06-2016 from http://factfinder.census.gov/faces/nav/jsf/pages/searchresults.xhtml?refresh=t
    dataDirectory = '../data/census/ACS_2013/'
    outputName = 'acs2013_clean.csv'
    
    if fromCache & os.path.isfile(dataDirectory + outputName):
        return pandas.read_csv(dataDirectory + outputName, header=0)
    
    else:
        # Retrieved 10-07-2016 from http://factfinder.census.gov/bkmk/table/1.0/en/ACS/13_5YR/B15003/0500000US04007.15000|0500000US04013.15000|0500000US04021.15000|0500000US04025.15000
        education = pandas.read_csv(dataDirectory + 'educational_attainment/ACS_13_5YR_B15003_with_ann.csv', 
                                    header=1)
        education['highschool'] = (education['Estimate; Total: - Regular high school diploma'] + 
                                   education['Estimate; Total: - GED or alternative credential']
                                  )
        education['undergraduate'] = (education['Estimate; Total: - Associate\'s degree'] +
                                      education['Estimate; Total: - Bachelor\'s degree']
                                     )
        education['postgraduate'] = (education['Estimate; Total: - Master\'s degree'] +
                                     education['Estimate; Total: - Professional school degree'] +
                                     education['Estimate; Total: - Doctorate degree']
                                    )
        # Retrieved 10-07-2016 from http://factfinder.census.gov/bkmk/table/1.0/en/ACS/13_5YR/B25077/0500000US04007.15000|0500000US04013.15000|0500000US04021.15000|0500000US04025.15000
        homeValue = pandas.read_csv(dataDirectory + 'median_home_value/ACS_13_5YR_B25077_with_ann.csv',  
                                    header=1)
        # Retrieved 10-07-2016 from http://factfinder.census.gov/bkmk/table/1.0/en/ACS/13_5YR/B19013/0500000US04007.15000|0500000US04013.15000|0500000US04021.15000|0500000US04025.15000
        income = pandas.read_csv(dataDirectory + 'median_household_income/ACS_13_5YR_B19013_with_ann.csv',  
                                 header=1)

        selectedData = (pandas.DataFrame(homeValue[['Id2', 'Estimate; Median value (dollars)']])
                        .merge(education[['Id2',
                                          'highschool',
                                          'undergraduate',
                                          'postgraduate'
                                        ]],
                               on='Id2', how='outer'
                             )
                        .merge(income[['Id2', 
                                       'Estimate; Median household income in the past 12 months (in 2013 inflation-adjusted dollars)'
                                      ]],
                               on='Id2', how='outer'                               
                              )
                        .rename(columns={'Id2': 'GEOID',
                                         'Estimate; Median value (dollars)': 'median_home_value',
                                         'highschool': 'education_highschool',
                                         'undergraduate': 'education_undergraduate',
                                         'postgraduate': 'education_postgraduate',
                                         'Estimate; Median household income in the past 12 months (in 2013 inflation-adjusted dollars)': 
                                         'median_household_income'
                                        }
                               )
                        )
        
        # remove character artifacts from income data        
        selectedData['median_home_value'] = pandas.to_numeric(
            selectedData['median_home_value']
            .str.strip('><+-')
            .str.replace(',', '')
        )
        
        selectedData['median_household_income'] = pandas.to_numeric(
            selectedData['median_household_income']
            .str.strip('><+-')
            .str.replace(',', '')
        )
        
        selectedData.fillna(selectedData.mean(), inplace=True)
    
        selectedData.to_csv(dataDirectory + outputName)
        return selectedData